In [19]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from imutils import paths
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import GlobalAveragePooling2D, Dense
from keras.losses import sparse_categorical_crossentropy

In [5]:
dataset = "C:\\Users\\T-GAMER\\Downloads\\base_dados"

In [6]:
imagePaths=list(paths.list_images(dataset))

In [7]:
data=[]
labels=[]

for i in imagePaths:
    label=i.split(os.path.sep)[-2]
    labels.append(label)
    image = load_img(i,target_size=(96,96))
    image = img_to_array(image)
    image = preprocess_input(image)
    data.append(image)

C:\Users\T-GAMER\miniconda3\envs\treinamento\lib\site-packages\PIL\Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [8]:
data = np.array(data, dtype='float32')
labels = np.array(labels)


In [9]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)


In [10]:
## Splitting of Data

train_X,test_X,train_Y,test_Y = train_test_split(data,labels,test_size=0.20,random_state=10,stratify=labels)

In [11]:
aug=ImageDataGenerator(
                       rotation_range=20,
                       zoom_range=0.15,width_shift_range=0.2,
                       height_shift_range=0.2,shear_range=0.15,
                       horizontal_flip=True,
                       vertical_flip=True,
                       fill_mode='nearest'
                       )

In [14]:
input_shape = (224,224,3)

In [21]:
# Build Model BEST MODEL
input_image = Input(shape=input_shape)
# 1st Conv layer
model = tf.keras.applications.MobileNetV3Small()
model = Sequential([model,Flatten(),Dense(1024),Dense(64),Dropout(0.2),Dense(2, activation='softmax')])




In [22]:
lr=0.0005
Epochs=100
BS=32

opt=Adam(learning_rate=lr,decay=lr/Epochs)

In [23]:
strategy = tf.distribute.MultiWorkerMirroredStrategy()
with strategy.scope():
    model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])

In [ ]:
history=model.fit(
aug.flow(train_X,train_Y,batch_size=BS),
steps_per_epoch=len(train_X)//BS,
validation_data=(test_X,test_Y),
validation_steps=len(test_X)//BS,
epochs=Epochs

)

Epoch 1/100
102/102 [==============================] - 83s 773ms/step - loss: 0.3226 - accuracy: 0.8652 - val_loss: 0.5388 - val_accuracy: 0.8523
Epoch 2/100
102/102 [==============================] - 84s 826ms/step - loss: 0.1416 - accuracy: 0.9546 - val_loss: 0.4469 - val_accuracy: 0.8755
Epoch 3/100
102/102 [==============================] - 78s 763ms/step - loss: 0.0807 - accuracy: 0.9750 - val_loss: 0.5875 - val_accuracy: 0.8791
Epoch 4/100
102/102 [==============================] - 79s 777ms/step - loss: 0.0799 - accuracy: 0.9747 - val_loss: 0.3259 - val_accuracy: 0.9243
Epoch 5/100
102/102 [==============================] - 78s 763ms/step - loss: 0.0693 - accuracy: 0.9787 - val_loss: 0.2020 - val_accuracy: 0.9609
Epoch 6/100
102/102 [==============================] - 88s 860ms/step - loss: 0.0548 - accuracy: 0.9778 - val_loss: 0.1450 - val_accuracy: 0.9683
Epoch 7/100
102/102 [==============================] - 80s 785ms/step - loss: 0.0456 - accuracy: 0.9836 - val_loss: 0.0846 -